In [1]:
import sys
sys.path.append("..")

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

import numpy as np
import pandas as pd
from typing import List
from tabulate import tabulate
from dataclasses import dataclass, field

# Trainer
from src.trainer.trainer import Trainer

# Models
from src.models.mlp import MLP
from src.models.p4_allcnn import P4AllCNNC
from src.models.fA_p4_allcnn import fA_P4AllCNNC
from src.models.p4m_allcnn import  P4MAllCNNC
from src.models.fA_p4m_allcnn import fA_P4MAllCNNC
from src.models.p4m_resnet import  P4MResNet
from src.models.fA_p4_resnet import fA_P4MResNet


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.5 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/larabastos/miniconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/larabastos/miniconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/larabastos/miniconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
 

AttributeError: _ARRAY_API not found

## CIFAR10 dataset

In [2]:
root_path = "../data"
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor()
])
transform_test = transforms.ToTensor()

full_train = datasets.CIFAR10(root=root_path, train=True, download=True, transform=transform_train)

train_size = int(0.9 * len(full_train))
val_size = len(full_train) - train_size
train_set, val_set = random_split(full_train, [train_size, val_size])
test_set = datasets.CIFAR10(root=root_path, train=False, download=True, transform=transform_test)


batch_size = 128
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

## Experiment

In [4]:
# ----- Helper Functions -----
def init_model(name):
    if name == "p4_allcnn":
        return P4AllCNNC()
    elif name == "fA_p4_allcnn":
        return fA_P4AllCNNC()
    elif name == "p4m_allcnn":
        return P4MAllCNNC()
    elif name == "fA_p4m_allcnn":
        return fA_P4MAllCNNC()
    elif name == "p4m_resnet":
        return P4MResNet()
    elif name == "fA_p4_resnet":
        return fA_P4MResNet()
    else:
        raise ValueError(f"Unknown model name: {name}")

def init_optimizer(model, lr, weight_decay):
    return optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)

def init_scheduler(optimizer, milestones):
    return optim.lr_scheduler.MultiStepLR(optimizer, milestones=[200, 250, 300], gamma=0.1)

# ----- Helper Classes -----
@dataclass
class HyperParams:
    lr: float
    epochs: int
    weight_decay: float
    momentum: float
    gamma: float
    milestones: List[int] = field(default_factory=list)

In [5]:
# ----- Configuration -----
num_iterations = 1
log_dir = "../logs"

model_hyperparameters = {
    "p4_allcnn":  HyperParams(lr=0.01, epochs=3, weight_decay=1e-3, momentum=0.9, milestones=[200, 250, 300], gamma=0.1),
    "fA_p4_allcnn": HyperParams(lr=0.01, epochs=3, weight_decay=1e-3, momentum=0.9, milestones=[200, 250, 300], gamma=0.1),
    #"p4m_allcnn": HyperParams(lr=0.01, epochs=1, weight_decay=1e-3, momentum=0.9, milestones=[200, 250, 300], gamma=0.1),
    #"fA_p4m_allcnn": HyperParams(lr=0.01, epochs=1, weight_decay=1e-3, momentum=0.9, milestones=[200, 250, 300], gamma=0.1),
    "p4m_resnet": HyperParams(lr=0.05, epochs=3, weight_decay=0.0, momentum=0.9, milestones=[50, 100, 150], gamma=0.1),
    # "fA_p4_resnet": HyperParams(lr=0.05, epochs=1, weight_decay=0.0, momentum=0.9, milestones=[50, 100, 150], gamma=0.1)
}
model_names = model_hyperparameters.keys()
accuracies = {name: [] for name in model_names}

In [6]:
# ----- Main Training Loop -----
for it in range(num_iterations):
    print(f"Iteration {it + 1}/{num_iterations}")

    for name in model_names:
        print(f"\n→ Training model: {name}")

        # 1. Grab hyperparams for this model
        hp = model_hyperparameters[name]

        # 2. Initialize model, criterion, optimizer, scheduler
        model = init_model(name)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(
            model.parameters(),
            lr=hp.lr,
            momentum=hp.momentum,
            weight_decay=hp.weight_decay
        )
        scheduler = optim.lr_scheduler.MultiStepLR(
            optimizer,
            milestones=hp.milestones,
            gamma=hp.gamma
        )

        # 3. Wrap in our Trainer (single‐model)
        trainer = Trainer(
            models={name: model},
            optimizers=[optimizer],
            criterions=[criterion],
            schedulers=[scheduler],
            log_dir=f"{log_dir}/{name}"
        )

        # 4. Train & validate with the model‐specific epoch count
        trainer.train(
            num_epochs=hp.epochs,
            train_loader=train_loader,
            val_loader=val_loader,
        )

        # 5. Evaluate on test set and record accuracy
        test_acc = trainer.evaluate(test_loader=test_loader)[name]
        accuracies[name].append(test_acc)

Iteration 1/1

→ Training model: p4_allcnn
Using device: cuda
Current GPU: NVIDIA GeForce RTX 4070 Laptop GPU
Training model : p4_allcnn


Training Epochs: 100%|████████████████████████████████████████████████████████████████████| 3/3 [01:20<00:00, 26.68s/it, Train Acc=0.4667, Val Acc=0.3728, Train Loss=1.5844, Val Loss=1.8259]



→ Training model: fA_p4_allcnn
Using device: cuda
Current GPU: NVIDIA GeForce RTX 4070 Laptop GPU
Training model : fA_p4_allcnn


Training Epochs: 100%|████████████████████████████████████████████████████████████████████| 3/3 [02:44<00:00, 54.68s/it, Train Acc=0.4694, Val Acc=0.3432, Train Loss=1.5532, Val Loss=1.8732]



→ Training model: p4m_resnet
Using device: cuda
Current GPU: NVIDIA GeForce RTX 4070 Laptop GPU
Training model : p4m_resnet


Training Epochs: 100%|████████████████████████████████████████████████████████████████████| 3/3 [04:03<00:00, 81.15s/it, Train Acc=0.6279, Val Acc=0.4970, Train Loss=1.0459, Val Loss=1.7660]


In [7]:
# ----- Final Statistics -----
final_stats = {
    name: {
        "% Test error": (1 - float(np.mean(vals))) * 100,
        "% std": float(np.std(vals)) * 100,
        "Num Parameters": sum(p.numel() for p in init_model(name).parameters())
    }
    for name, vals in accuracies.items()
}

## Table generation

In [8]:
df = pd.DataFrame.from_dict(final_stats, orient='index')
df = df.round(2)
print("📊 Model Accuracy Summary in CIFAR10\n")
print(tabulate(df, headers="keys", tablefmt="fancy_grid"))

📊 Model Accuracy Summary in CIFAR10

╒══════════════╤════════════════╤═════════╤══════════════════╕
│              │   % Test error │   % std │   Num Parameters │
╞══════════════╪════════════════╪═════════╪══════════════════╡
│ p4_allcnn    │          64.38 │       0 │      1.37037e+06 │
├──────────────┼────────────────┼─────────┼──────────────────┤
│ fA_p4_allcnn │          65.76 │       0 │      1.40012e+06 │
├──────────────┼────────────────┼─────────┼──────────────────┤
│ p4m_resnet   │          50.89 │       0 │      2.62168e+06 │
╘══════════════╧════════════════╧═════════╧══════════════════╛
